<a href="WhatIsBuildmachine"></a>
<h2 style="color:blue">What's the difference? `icarusbuild01` vs. `icarusgpvm0X`</h2>

There are multiple machines you can `ssh` into for your development: `icarusbuild01` and `icarusgpvm0X` where X is a positive integer (1 to some number). What are the differences?

* Among all machines, there are shared, network-mounted disk spaces `/icarus/data` and `/icarus/app`. The former is meant to use for a temporary, small data files, and the latter is meant to be used to build a software (i.e. applications). 


* For `icarusbuild01`, there's an additional `/scratch/icarus` space made available in the form of a local, physical disk. A local disk space has much faster disk IO than the network mounted spaces such as `/icarus/data` and `/icarus/app`, and this makes `icarusbuild01` a much better machine for building a software, which involves many small file read and write (IO). 


* `icarusgpvm0X` machines are all *virtual machines*, which means they are virtual machine applications running on physical machines. It is typical for multiple virtual machine instances running on one physical machine including virtual machines (i.e. *gpvm*s) of other experiments at Fermilab (SBND, MicroBooNE, DUNE, g-2, etc. etc.). As a result, virtual machines can be slow because multiple instances may be sharing one physical machine, and you cannot check what processes are running on other virtual machines that are on the same physical machine. On the other hand, `icarusbuild01` is a physical server and not a virtual machine. You can see what other processes are running to know when the machine is busy, and also often times the load is smaller compared to `gpvm`s as it is a dedicated instance. This is another reason for some people to favor `icarusbuild01` over `gpvm`s for software compilation and small test purposes.


<a href="WhatIsPNFS"></a>
<h2 style="color:blue">What is `/pnfs`?</h2>

Fermilab `/pnfs` is a cache disk (name)space managed by Fermilab using [dCache](https://www.dcache.org/). The space you should be using for ICARUS is under `/pnfs/icarus/scratch` and `/pnfs/icarus/persistent`. Some points noted below.


* Use `/pnfs/icarus/scratch` to *stage* files, meaning make files available temporarily to/from grid jobs. 


* Use `/pnfs/icarus/persistent` to *store* important files that should not be removed but expect frequent access. Very few files should meet this criteria, and if you think "oh I should store my files here", be cautious: often that is not true. For a permanent storage of files with infrequent access, you should use the tape storage system which can be easily linked to `/pnfs` space (i.e. synchronize to make files automatically available from tape to the cache area) using [SAM](https://cdcvs.fnal.gov/redmine/projects/sam/wiki/User_Guide_for_SAM) file database system (yep, Fermilab eco-system!). 


* An interactive access to files stored under `/pnfs` is discouraged. If you want to do, say, an interactive data analysis, download the files.


<a href="WhatIsUPS"></a>
<h2 style="color:blue">What is `ups`?</h2>

In short, it is a Fermilab-specific software package management system, so kind of like `yum`, `apt`, or `pip`. The difference is that it's only useful within Fermilab experiments. That's a huge difference. If you want to read details, here is [a 39 chapters long manual](https://cdcvs.fnal.gov/redmine/projects/ups/wiki/ReferenceManual) <style "color:red">(no I've never read, in fact, I just discovered)</style>. Some `ups` commands are useful for your daily development lives and worth noting below.

*  `ups list -aK+` will list ALL packages and versions that are available for you to set up.
  * `ups list -aK+ X` runs MUCH faster to list available versions of `X` package (for example, `icaruscode`). So when you just want to list versions, always specify the package name rather than listing all thousands available.


* `ups setup X VERSION [-q FLAVOR]` will setup a software `X` with the specified VERSION and optionally a FLAVOR. For example, `ups setup icaruscode v08_30_01 -q e17:prof` will set-up `icaruscode` with a version `v08_30_01` built with a flavor `e17:prof`.


* `ups active` will list ALL packages that you have `ups setup` so far.


* `ups depend X VERSION [-q FLAVOR]` will list ALL softwares that depend on `X` with the specified version and (optionally) a flavor.
  * e.g.) try `ups depend icaruscode v08_30_01 -q e17:prof`

<a href="WhatIsMRB"></a>
<h2 style="color:blue">What is `mrb`?</h2>

`mrb` stands for multi-repository build, another Fermilab specific software that works with `ups` to allow you to develop and build multiple software repositories together. For a software to be built with `mrb`, it has to have `mrb`-compatible structures **and** it needs to be checked-out via `mrb`-way (i.e. no, you typing a plain `git clone` won't work). Fortunately, for almost everyone, you do not need to understand all details about `mrb`-compatibleness nor `mrb`-way, though! Some useful tips are noted below.

* Shell environment variables!
  * `$MRB_TOP` points to the very top directory of your development area
  * `$MRB_SOURCE` points to where you download software repositories to develop
  * `$MRB_BUILDDIR` points to where the build outputs are produced
  * `$MRB_INSTALL` points to where the built products are *installed* (library, headers, etc.). This is the directory you want to `tar` for a batch job submission
  
  
* `mrbsetenv` will check the dependent software versions to identify any conflict, and complete `ups setup` those softwares if no error (so you don't have to `ups setup` gazillions by yourself).


* `mrb g X` will `git checkout` the software `X`. For example, you can try `mrb g icaruscode`.


* `mrb i -j4` will *build and install* checked out softwares (under `$MRB_SOURCE`) using 4 parallel processes (`-j4` flag).


<a href="HowMRBWorks"></a>
<h2 style="color:blue">How does `mrb` figure out softare version dependnecies?</h2>


As explained in [above](#WhatIsMRB), `mrbsetenv` checks software dependencies with versions. How does it work? First of all, `mrb` works with `ups`, a package management system. When you want to build your locally checked out softwares, which is why you are using `mrb`, each software carries a formatted text file that denote the dependent softwares with versions. You can find this under `X/ups/product_deps` text file where `X` is your software name. For example, for `icaruscode`, it is under `icaruscode/ups/product_deps`. This is one of *`mrb`-compatible structure* mentioned above. If you look at this text file, it lists `ups` products that need to be set-up.

* If you need to introduce a new dependency for your development, modify `ups/product_deps` file.


* If `ups/product_deps` file specifies a software with a version that conflicts with other softwares/versions already made available for you (i.e. via `ups setup`), then `mrbsetenv` will give you an error message.

  * In other words, if you get an error upon `mrbsetenv`, you can modify this file to work around.
  
  
<a href="WhatIsEvent"></a>
<h2 style="color:blue">What is an *event*?</h2>



<a href="WhatIsModule"></a>
<h2 style="color:blue">What is *art module* (`EDProducer`, `EDAnalyzer`, and `EDFilter`)?</h2>

You can read all about *art*, an event processing framework developed at Fermilab, [here](https://cdcvs.fnal.gov/redmine/projects/art/wiki). But I try to give a short description below.

* `art` module is a unit of event processing. In a typical data analysis/reconstruction/simulation workflow, there's a concept of an *event* ([see above](#WhatIsEvent) to learn a concept of event). Event is a unit: you want to simulate, reconstruct, and analyze an event. Naturally, you might want to apply the same algorithms for all events in a file (i.e. an algorithm to *"find a neutrino interaction"*). In addition, you might want a freedom to combine multiple algorithms at run time (i.e. combine two algorithms: *"reject cosmic ray backgrounds"* + *"find a neutrino interaction"*). We call each of these actions (algorithms) *a process*. The capability we want is to write code to define a process, and configure to loop over many events to automatically apply these processes. This is what an event processing framework like `art` can do for you. You can program a process, a modularized piece of code, and configure to run many processes to run on many events easily using `art`. Of course, it won't come free: you have to follow the procedures `art` specifies, such as a type of process and how to write them. The types of processes are exactly `EDProducer`, `EDAnalyzer`, and `EDFilter`.


* `EDProducer` ... can *read* and *write* the data through `art::Event&` non-const reference object. If you want to create and store new data contents, you should use this module type.

  * Try `cetskelgen producer` to produce a template (empty), compilable `EDProducer` source code.


* `EDAnalyzer` ... can only *read* the data through `const art::Event&` const reference object. If you want to perform an analysis on stored data, you should use this module type.

  * Try `cetskelgen analyzer` to produce a template (empty), compilable `EDProducer` source code.


* `EDFilter` ... is just like `EDAnalyzer` but has a special functionality to alter the event processing path(s). For example, you can configure `art` to define 2 output streams: events that are classified as type X and other events that are classified as type Y, using `EDFilter` to return `true/false`. You can define the event processing configuration to, then, apply different process (`art` modules) on two split paths. This can be done arbitrary times to split the processing path into N ways. 

  * Try `cetskelgen filter` to produce a template (empty), compilable `EDProducer` source code.


<a href="WhatIsService"></a>
<h2 style="color:blue">What is `art::Service`?</h2>

How dare you ask that question (the answer will be filled)


<a href="HowToMakeModule"></a>
<h2 style="color:blue">How to create an `art` module?</h2>



<a href="HowToFindLArHeader"></a>
<h2 style="color:blue">How can I find WTF.h header file?</h2>

First of all, if it's coming from `ups` packages, you probably see some directory structures in the include path. For example, you might see  something like:
```
#include "larcore/Geometry/Geometry.h"
```
... and the prefix `larcore` is important: typically the first directory name refers to the software package name this exists (so that includes from multiple packages don't get confused). If you have `icaruscode` environment setup, you probably see:
```
-bash-4.1$ printenv | grep LARECORE_INC
```
In my case, with v08_30_01 version of `icaruscode`, this gives me:
```
$> printenv | grep LARCORE_INC
LARCORE_INC=/cvmfs/larsoft.opensciencegrid.org/products/larcore/v08_05_00/include
```
Now all you need to do is to concatenate:
```
bash-4.1$ ls $LARCORE_INC/larcore/Geometry/Geometry.h
/cvmfs/larsoft.opensciencegrid.org/products/larcore/v08_05_00/include/larcore/Geometry/Geometry.h
```

If you want to see ALL *include* paths, try:
```
bash-4.1$ printenv | grep INC=
```
If you want to find only ones for `larsoft` related, you could also add ` | grep LAR` :)

